# Import Packages

In [0]:
import warnings
warnings.filterwarnings("ignore")

#Basic
import pandas as pd
import gensim
import nltk
import re
import numpy as np
import math
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import rpy2.robjects.packages as rpackages

#For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#For Data Preparation
import itertools
from itertools import combinations

#For Class Definition
from sklearn.base import BaseEstimator, TransformerMixin

# Import Data

In [ ]:
 df = pd.read_csv('C:/Users/Louis Owen/Desktop/ICoDSA 2020/Dataset/df_prepared.csv')

# Data Cleaning

## Basic

In [0]:
df.head(2)

,id,created_at,body,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,caps_word,hashtags,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,PMI_score
0,149411955.0,2019-01-01 02:01:50+00:00,$BA thinking about buying a lot and hold it fo...,thinking about buying a lot and hold it for lo...,think about buy a lot and hold it for long ani...,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-0.770743,0.015570,-0.116739,0.142857,0.857143,0.000,0.000,0.00,0.000000,0.000000,4.987022,1.360259,9.147977,1.098612,10.173580,1.377608,10.210440,1.194506,-0.345702
1,149420817.0,2019-01-01 14:31:18+00:00,$BA $AAPL $NVDA $AMZN $NFLX $LMT $NOC So is it...,So is it safe to say that the market bottomed ...,So is it safe to say that the market bottom la...,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.2,0.2,0.0,0.05,0.0,0.05,0.0,0.0,0.0,-0.815584,0.024566,-0.137776,0.100000,0.900000,-0.125,0.125,-0.25,0.083333,0.083333,4.651765,1.473812,8.238657,1.249515,9.887319,1.098612,10.167119,1.098612,0.083216


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77025 entries, 0 to 77024
Data columns (total 67 columns):
id                            77025 non-null float64
created_at                    77025 non-null object
body                          77025 non-null object
clean_text                    77025 non-null object
base_text                     77022 non-null object
POS_VB                        77025 non-null int64
POS_VBD                       77025 non-null int64
POS_VBG                       77025 non-null int64
POS_VBN                       77025 non-null int64
POS_VBP                       77025 non-null int64
POS_VBZ                       77025 non-null int64
POS_VM                        77025 non-null int64
+num                          77025 non-null int64
-num                          77025 non-null int64
num%                          77025 non-null int64
+num%                         77025 non-null int64
-num%                         77025 non-null int64
$num              

## Missing Values Analysis

In [0]:
def missing_values(df):
  '''
  Function to check features with missing values
  '''
  missing_values_feat=[]
  for column in df.columns:
    if df[column].isnull().values.any():
      missing_values_feat.append(column)
  return(missing_values_feat)

missing_values(df)

['base_text',
 'AFINN_max_score',
 'AFINN_min_score',
 'AFINN_pos_ratio',
 'AFINN_neg_ratio',
 'BingLiu_pos_ratio',
 'BingLiu_neg_ratio',
 'General_Inquirer_pos_ratio',
 'General_Inquirer_neg_ratio',
 'NRC_Hashtag_max_score',
 'NRC_Hashtag_min_score',
 'NRC_Hashtag_pos_ratio',
 'NRC_Hashtag_neg_ratio',
 'SentiWordNet_max_score',
 'SentiWordNet_min_score',
 'Avg_TFIDF_1-grams',
 'Avg_TFIDF_2-grams',
 'Avg_TFIDF_3-grams',
 'Avg_TFIDF_4-grams']

In [0]:
len(df)

77025

In [0]:
df[['AFINN_max_score']].describe()

,AFINN_max_score
count,77022.000000
mean,0.169287
std,0.240960
min,-1.000000
25%,0.000000
50%,0.000000
75%,0.400000
max,1.000000


In [0]:
df[pd.isnull(df.AFINN_max_score)].head()

,id,created_at,body,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,caps_word,hashtags,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,PMI_score
63849,183293697.0,2019-11-11 20:21:49+00:00,$BA ....,....,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.203973,NaN,1.207333,NaN,1.098612,NaN,1.194506,-1.146964
66785,184857263.0,2019-11-24 23:19:58+00:00,$BA $IHI $INSP $NOW $ZM ..,..,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.456620,NaN,1.933662,NaN,1.194506,NaN,1.194506,-0.129754
74922,187988225.0,2019-12-24 16:01:30+00:00,$BA \n\n;),\n\n;),NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.456620,NaN,1.933662,NaN,1.194506,NaN,1.194506,-0.129754


Remove rows with nan affin max score, because the clean text is not proper.

In [0]:
missing_values(df[~pd.isnull(df.AFINN_max_score)])

['SentiWordNet_max_score',
 'SentiWordNet_min_score',
 'Avg_TFIDF_1-grams',
 'Avg_TFIDF_2-grams',
 'Avg_TFIDF_3-grams',
 'Avg_TFIDF_4-grams']

In [0]:
df[['SentiWordNet_max_score']].describe()

,SentiWordNet_max_score
count,74040.000000
mean,0.226614
std,0.247654
min,-0.875000
25%,0.000000
50%,0.125000
75%,0.375000
max,1.000000


In [0]:
df[pd.isnull(df.SentiWordNet_max_score)].head()

,id,created_at,body,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,caps_word,hashtags,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,PMI_score
2,149421896.0,2019-01-01 15:27:52+00:00,$SPY $QQQ $IWM $AAPL $GOOGL $AMZN $FB $BABA $C...,#NEVERFORGET,#neverforget,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.102069,-0.102069,-0.102069,0.000000,1.000000,0.0,NaN,NaN,0.0,0.0,NaN,1.574081,NaN,1.602930,NaN,1.290400,NaN,1.098612,-0.170713
38,149480984.0,2019-01-02 16:30:17+00:00,$ASTC -9.83 %! What happened?!!! https://blue...,-9.83 %! What hapened?!!! _url $BA,-9.83 %! what hapened?!!! _url $BA,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,4,1,1,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.083662,0.000000,-0.083662,0.000000,0.166667,0.0,NaN,NaN,0.0,0.0,5.479952,1.480349,10.007708,1.098612,10.21044,1.194506,NaN,0.963457,-0.091357
69,149540368.0,2019-01-03 00:22:19+00:00,$BA $$$_,$$$ _,$$$ _,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.171338,-0.063733,-0.107605,0.000000,1.000000,0.0,NaN,NaN,0.0,0.0,NaN,1.870704,NaN,1.714926,NaN,1.268887,NaN,1.194506,0.144486
78,149553487.0,2019-01-03 03:29:17+00:00,Thurs 👀 1 $AAPL $AMZN $FB $BABA $AMD $NFLX $MS...,Thurs 1 $GILD,thur 1 $GILD,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.024912,0.018061,-0.042973,0.333333,0.333333,0.0,NaN,NaN,0.0,0.0,8.130999,1.306085,NaN,1.130577,NaN,1.146559,NaN,1.560710,-0.641265
107,149597934.0,2019-01-03 16:03:58+00:00,$BA https://seekingalpha.com/news/3420404-boei...,_url,_url,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0,0.0,2.854160,1.456620,NaN,1.245890,NaN,1.290400,NaN,1.098612,-0.129754


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 

In [0]:
df[['SentiWordNet_min_score']].describe()

,SentiWordNet_min_score
count,74040.000000
mean,-0.228789
std,0.283739
min,-1.000000
25%,-0.500000
50%,-0.125000
75%,0.000000
max,1.000000


In [0]:
df[pd.isnull(df.SentiWordNet_min_score)].head()

,id,created_at,body,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,caps_word,hashtags,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,PMI_score
2,149421896.0,2019-01-01 15:27:52+00:00,$SPY $QQQ $IWM $AAPL $GOOGL $AMZN $FB $BABA $C...,#NEVERFORGET,#neverforget,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.102069,-0.102069,-0.102069,0.000000,1.000000,0.0,NaN,NaN,0.0,0.0,NaN,1.574081,NaN,1.602930,NaN,1.290400,NaN,1.098612,-0.170713
38,149480984.0,2019-01-02 16:30:17+00:00,$ASTC -9.83 %! What happened?!!! https://blue...,-9.83 %! What hapened?!!! _url $BA,-9.83 %! what hapened?!!! _url $BA,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,4,1,1,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.083662,0.000000,-0.083662,0.000000,0.166667,0.0,NaN,NaN,0.0,0.0,5.479952,1.480349,10.007708,1.098612,10.21044,1.194506,NaN,0.963457,-0.091357
69,149540368.0,2019-01-03 00:22:19+00:00,$BA $$$_,$$$ _,$$$ _,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.171338,-0.063733,-0.107605,0.000000,1.000000,0.0,NaN,NaN,0.0,0.0,NaN,1.870704,NaN,1.714926,NaN,1.268887,NaN,1.194506,0.144486
78,149553487.0,2019-01-03 03:29:17+00:00,Thurs 👀 1 $AAPL $AMZN $FB $BABA $AMD $NFLX $MS...,Thurs 1 $GILD,thur 1 $GILD,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.024912,0.018061,-0.042973,0.333333,0.333333,0.0,NaN,NaN,0.0,0.0,8.130999,1.306085,NaN,1.130577,NaN,1.146559,NaN,1.560710,-0.641265
107,149597934.0,2019-01-03 16:03:58+00:00,$BA https://seekingalpha.com/news/3420404-boei...,_url,_url,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0,0.0,2.854160,1.456620,NaN,1.245890,NaN,1.290400,NaN,1.098612,-0.129754


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 

In [0]:
df[['Avg_TFIDF_1-grams']].describe()

,Avg_TFIDF_1-grams
count,76360.000000
mean,5.659666
std,1.779062
min,2.166267
25%,4.967886
50%,5.505527
75%,6.144626
max,335.169234


In [0]:
df[pd.isnull(df['Avg_TFIDF_1-grams'])]

,id,created_at,body,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,caps_word,hashtags,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,PMI_score
2,149421896.0,2019-01-01 15:27:52+00:00,$SPY $QQQ $IWM $AAPL $GOOGL $AMZN $FB $BABA $C...,#NEVERFORGET,#neverforget,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.102069,-0.102069,-0.102069,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,1.574081,NaN,1.602930,NaN,1.290400,NaN,1.098612,-0.170713
69,149540368.0,2019-01-03 00:22:19+00:00,$BA $$$_,$$$ _,$$$ _,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.171338,-0.063733,-0.107605,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,1.870704,NaN,1.714926,NaN,1.268887,NaN,1.194506,0.144486
166,149676509.0,2019-01-04 03:40:49+00:00,Friday’s👀1 $AAPL $AMZN $AMD $FB $GOOGL $MSFT $...,Fridays1 $ADBE,fridays1 $ADBE,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.526225,NaN,1.521449,NaN,1.194506,NaN,1.656604,-0.432726
172,149690340.0,2019-01-04 12:34:58+00:00,$SPY $QQQ $GOOG $GOOGL $BA $DIA,$DIA,$DIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.217948,NaN,1.386294,NaN,1.059351,NaN,1.194506,0.209632
212,149774000.0,2019-01-04 21:35:55+00:00,$BABA $FB $SQ $MSFT $NVDA $NFLX $AMZN $BA $SPY...,$STUDY,$STUDY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.217948,NaN,1.386294,NaN,1.059351,NaN,1.194506,0.209632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75945,188202027.0,2019-12-27 15:22:30+00:00,$BA 💰👍🏻.,💰👍🏻.,💰👍🏻.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.456620,NaN,1.933662,NaN,1.098612,NaN,1.194506,-0.129754
75972,188203848.0,2019-12-27 15:31:15+00:00,$BA 350c jan17🦏,350c jan17,350c jan17,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,1.747066,NaN,1.327591,NaN,1.098612,NaN,1.194506,0.575152
76004,188213455.0,2019-12-27 16:20:12+00:00,$BA $SPY,$SPY,$SPY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.071206,-0.071206,-0.071206,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,1.403355,NaN,1.244524,NaN,1.194506,NaN,1.194506,0.036319
76510,188403697.0,2019-12-30 16:24:13+00:00,$BA $300,$300,$300,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.071206,-0.071206,-0.071206,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,1.300441,NaN,1.684921,NaN,1.194506,NaN,1.464816,-0.428078


Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value

## Missing Values Imputation

Missing Value Actionable Items:
1.   Remove rows with nan [AFINN_max_score], because the clean text is not proper.
2.   Impute 0 for [SentiWordNet_max_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
3.   Impute 0 for [SentiWordNet_min_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
4.   Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value






In [0]:
df=df[~pd.isnull(df.AFINN_max_score)]
df['SentiWordNet_max_score']=df['SentiWordNet_max_score'].fillna(0)
df['SentiWordNet_min_score']=df['SentiWordNet_min_score'].fillna(0)
df['Avg_TFIDF_1-grams']=df['Avg_TFIDF_1-grams'].fillna(0)
df['Avg_TFIDF_2-grams']=df['Avg_TFIDF_2-grams'].fillna(0)
df['Avg_TFIDF_3-grams']=df['Avg_TFIDF_3-grams'].fillna(0)
df['Avg_TFIDF_4-grams']=df['Avg_TFIDF_4-grams'].fillna(0)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77022 entries, 0 to 77024
Data columns (total 67 columns):
id                            77022 non-null float64
created_at                    77022 non-null object
body                          77022 non-null object
clean_text                    77022 non-null object
base_text                     77022 non-null object
POS_VB                        77022 non-null int64
POS_VBD                       77022 non-null int64
POS_VBG                       77022 non-null int64
POS_VBN                       77022 non-null int64
POS_VBP                       77022 non-null int64
POS_VBZ                       77022 non-null int64
POS_VM                        77022 non-null int64
+num                          77022 non-null int64
-num                          77022 non-null int64
num%                          77022 non-null int64
+num%                         77022 non-null int64
-num%                         77022 non-null int64
$num              

## Final Touch

In [0]:
df['created_date']=df['created_at'].apply(lambda x: pd.to_datetime(x).date())
df['created_time']=df['created_at'].apply(lambda x: pd.to_datetime(x).time())
df['time']=df['created_time'].apply(lambda x: 1 if (x<pd.to_datetime('09:30:00').time()) else 2 if (x<pd.to_datetime('10:30:00').time()) else
                         3 if (x<pd.to_datetime('11:30:00').time()) else 4 if (x<pd.to_datetime('12:30:00').time()) else 5 if (x<pd.to_datetime('13:30:00').time()) else
                         6 if (x<pd.to_datetime('14:30:00').time()) else 7 if (x<pd.to_datetime('15:30:00').time()) else 1)

In [0]:
df=df[['created_date',
 'time',
 'clean_text',
 'base_text',d
 'POS_VB',
 'POS_VBD',
 'POS_VBG',
 'POS_VBN',
 'POS_VBP',
 'POS_VBZ',
 '+num',
 '-num',
 '+num%',
 '-num%',
 'word_num',
 'ordinal_num',
 'num-num',
 'num-num%',
 'num-num-num',
 'num/num',
 'only_number',
 'call_+num%',
 'put_+num%',
 'put_-num%',
 'bull',
 'bear',
 'number_of_!',
 'number_of_?',
 'number_of_$',
 'continous_!',
 'continous_?',
 'AFINN_sum_score',
 'AFINN_max_score',
 'AFINN_min_score',
 'AFINN_pos_ratio',
 'AFINN_neg_ratio',
 'BingLiu_pos_ratio',
 'BingLiu_neg_ratio',
 'NRC_Hashtag_sum_score',
 'NRC_Hashtag_max_score',
 'NRC_Hashtag_min_score',
 'NRC_Hashtag_pos_ratio',
 'NRC_Hashtag_neg_ratio',
 'SentiWordNet_sum_score',
 'SentiWordNet_max_score',
 'SentiWordNet_min_score',
 'SentiWordNet_pos_ratio',
 'SentiWordNet_neg_ratio',
 'Avg_TFIDF_1-grams',
 'Avg_rf_1-grams',
 'Avg_TFIDF_2-grams',
 'Avg_rf_2-grams',
 'Avg_TFIDF_3-grams',
 'Avg_rf_3-grams',
 'Avg_TFIDF_4-grams',
 'Avg_rf_4-grams',
 'caps_word',
 'hashtags',
 'PMI_score']]

In [0]:
df

,created_date,time,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,+num,-num,+num%,-num%,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,only_number,call_+num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,caps_word,hashtags,PMI_score
0,2019-01-01,1,thinking about buying a lot and hold it for lo...,think about buy a lot and hold it for long ani...,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,-0.770743,0.015570,-0.116739,0.142857,0.857143,0.000,0.000,0.000,0.000000,0.000000,4.987022,1.360259,9.147977,1.098612,10.173580,1.377608,10.210440,1.194506,0,0,-0.345702
1,2019-01-01,7,So is it safe to say that the market bottomed ...,So is it safe to say that the market bottom la...,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.2,0.2,0.0,0.050000,0.000000,0.05,0.000000,-0.815584,0.024566,-0.137776,0.100000,0.900000,-0.125,0.125,-0.250,0.083333,0.083333,4.651765,1.473812,8.238657,1.249515,9.887319,1.098612,10.167119,1.098612,0,0,0.083216
2,2019-01-01,7,#NEVERFORGET,#neverforget,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,-0.102069,-0.102069,-0.102069,0.000000,1.000000,0.000,0.000,0.000,0.000000,0.000000,0.000000,1.574081,0.000000,1.602930,0.000000,1.290400,0.000000,1.098612,1,1,-0.170713
3,2019-01-01,1,Helo where do you see going in the coming days...,helo where do you see go in the come day week ...,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4,0.4,0.0,0.076923,0.000000,0.00,0.000000,-0.528960,0.041589,-0.122137,0.153846,0.769231,0.125,0.125,0.000,0.142857,0.000000,5.228812,1.465099,8.746664,1.109602,9.750493,1.098612,10.037718,1.098612,0,0,-0.218591
4,2019-01-01,1,ripping -9.83 % _url $BA,rip -9.83 % _url $BA,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,-0.267663,0.000000,-0.162826,0.000000,0.400000,-0.250,-0.250,-0.250,0.000000,1.000000,5.839153,1.277616,10.210440,1.194506,0.000000,1.425555,0.000000,1.560710,0,0,0.760171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77020,2019-12-31,1,260$in 2020 can not wait,260$in 2020 can not wait,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,-0.917584,0.010311,-0.733444,0.200000,0.600000,-0.625,0.000,-0.625,0.000000,0.500000,4.163501,1.394412,6.161358,1.175688,7.360439,1.098612,0.000000,1.098612,0,0,-0.318860
77021,2019-12-31,1,NEW ARTICLE : Year in Aerospace: Looking back ...,new articl : year in aerospace: look back at b...,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.4,0.0,-0.4,0.000000,0.038462,0.00,0.038462,-0.497128,0.055429,-0.092520,0.192308,0.653846,0.625,0.375,-0.250,0.200000,0.066667,5.204233,1.486436,8.007619,1.242453,8.788098,1.207593,8.967392,1.098612,2,0,-0.253761
77022,2019-12-31,1,Small portfolio proposal: AutoZone($AZO) Boein...,small portfolio proposal: autozone($azo) boein...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,-0.326413,0.000000,-0.279358,0.000000,0.272727,-0.375,0.000,-0.375,0.000000,0.333333,7.286005,1.188114,10.436985,1.310031,10.768251,1.098612,10.826174,1.098612,3,2,-0.065637
77023,2019-12-31,1,Turkish Airlines says Boeing agrees to compens...,

# Export

In [0]:
df.to_csv('C:/Users/Louis Owen/Desktop/ICoDSA 2020/SENN/Dataset/Final/df_stocktwits_prepared_final.csv',index=False)